#Mounting Google Drive

In [18]:
from google.colab import drive
drive.mount('/content/drive')
base_path = "/content/drive/MyDrive/FDI Projects"
raw_file = f"{base_path}/Sectoral FDI Equity Data (25 years).xlsx"
db_path = f"{base_path}/fdi_sector.db"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Data Maker

In [6]:
import pandas as pd

In [7]:
def make_clean_data(file_path: str) -> pd.DataFrame:
    df = pd.read_excel(file_path)
    df = df[~df['Sector'].astype(str).str.contains("Total|Sumcheck", na=False)]
    df.columns = [col.strip().replace(" ", "_").lower() for col in df.columns]
    df = df.melt(id_vars='sector', var_name='year', value_name='inflow_mn_usd')

    df['year'] = df['year'].astype(str)
    df['inflow_mn_usd'] = pd.to_numeric(df['inflow_mn_usd'], errors='coerce')
    df = df.dropna(subset=['inflow_mn_usd'])

    return df

#Data Reader

In [19]:
clean_df = make_clean_data(raw_file)
clean_df.head()

,sector,year,inflow_mn_usd
0,METALLURGICAL INDUSTRIES,2000-01,22.69
1,MINING,2000-01,1.32
2,POWER,2000-01,89.42
3,NON-CONVENTIONAL ENERGY,2000-01,0.00
4,COAL PRODUCTION,2000-01,0.00


#Data Feeder

In [20]:
import sqlite3

In [21]:
def feed_to_sqlite(df: pd.DataFrame, db_path: str, table_name: str):
    conn = sqlite3.connect(db_path)
    df.to_sql(table_name, con=conn, if_exists='replace', index=False)
    conn.close()

In [22]:
feed_to_sqlite(clean_df, db_path, "fdi_sector")